# Dataset Grab and Structure

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

ds = load_dataset("InferencePrince555/Resume-Dataset")

In [ ]:
ds

In [ ]:
df = ds['train'].to_pandas()
swe_df = df[df['instruction'].str.contains("Generate a Resume for a Software Developer Job", case=False, na=False)]
network_df = df[df['instruction'].str.contains("Generate a Resume for a Network Administrator Job", case=False, na=False)]
pm_df =  df[df['instruction'].str.contains("Generate a Resume for a Project manager Job", case=False, na=False)]

In [ ]:
import pandas as pd

combined_df = pd.concat([swe_df, network_df, pm_df], ignore_index=True)

In [ ]:
combined_df

In [ ]:
combined_df['instruction'] = combined_df['instruction'].map({'Generate a Resume for a Software Developer Job': 1, 'Generate a Resume for a Project manager Job': 2, 'Generate a Resume for a Network Administrator Job':3})
final_df = combined_df.drop('input', axis=1)

In [ ]:
final_df

# Embedded Vectorization of Resume Text

In [ ]:
pip install transformers torch scikit-learn 


In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[0][0].numpy()

X = final_df['Resume_test'].apply(get_bert_embedding).tolist()

y = final_df['instruction'] 

y = y.apply(lambda label: 1 if label == 1 else 0)




# Convert to CSV 

In [ ]:
df = pd.DataFrame({'embedded': X, 'label': y})

In [ ]:
df.to_csv('cleaned_data.csv')